# スタッキングの実装
ここでは2値分類用のスタッキングのクラスを実装する。  
scikit-learnにもスタッキング用のクラスが存在するが、学習時にKFold CVをしていない模様。なので過学習気味になる恐れ。    
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html

本ドキュメントの参考資料：  
[【本番編!!】米国データサイエンティストがやさしく教える機械学習超入門【Pythonで実践】](https://www.udemy.com/share/108nHI3@xpAI18mdRm1C_i4744C1DGbEtA6OMBG1WO06UV5L6j73HgS7l7uap7-gtqM2l5bf/)

In [159]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, label_binarize, OrdinalEncoder
# import statsmodels.api as sma
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.model_selection import train_test_split ,LeaveOneOut, cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster
from sklearn import tree
from xgboost import XGBClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier

%matplotlib inline
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
# polarsでタイタニックデータを読み込み
df = pl.from_pandas(sns.load_dataset('titanic'))

# GBDTは欠損値の対処は不要だが、決定木やランダムフォレストは対処が必要。今回は単純に落とす。
df = df.drop_nulls()

# 学習データ、目的変数を定義
X = df.drop(['survived', 'alive'])
y = df.get_column('survived')

# カテゴリ変数のカラム名をリスト化
category_cols = X.select(pl.col([pl.Utf8, pl.Categorical, pl.Boolean])).columns

# ラベルエンコーディング（LabelEncoderではなく、OrdinalEncoderを使う）
oe = OrdinalEncoder()
# pandasで返ってくるように指定。polarsは指定できない模様
oe.set_output(transform='pandas')
# カテゴリ変数をエンコーディング。polars.DFはそのまま入れられないのでpandasに変換する。
X = X.with_columns( pl.from_pandas(oe.fit_transform(X.select(category_cols).to_pandas())) )

# hold-out
X_train, X_test, y_train, y_test = train_test_split(X.to_pandas(), y.to_pandas(), test_size=0.3, random_state=0)

## 実装前の確認

In [121]:
# kfoldの定義
cv = KFold(n_splits=5, shuffle=True, random_state=0)
estimator = XGBClassifier(early_stopping_rounds=10, learning_rate=0.01, eval_metric='auc',random_state=0)

In [122]:
%%time
X_pd = X.to_pandas()
y_pd = y.to_pandas()
y_pred_proba_li = []

# Layer1
for train_index, val_index in cv.split(X_pd):
    X_train, X_val = X_pd.iloc[train_index], X_pd.iloc[val_index]
    y_train, y_val = y_pd.iloc[train_index], y_pd.iloc[val_index]
    
    # モデル学習
    eval_set = [(X_val, y_val)]
    estimator.fit(X_train, y_train, eval_set=eval_set, verbose=True)
    # 検証データ(学習に使っていないデータ)に対する予測値算出
    y_pred_proba = estimator.predict_proba(X_val)

    # 予測値を追加していく
    y_pred_proba_li.append(y_pred_proba)

result = np.concatenate(y_pred_proba_li)

[0]	validation_0-auc:0.92778
[1]	validation_0-auc:0.93704
[2]	validation_0-auc:0.94074
[3]	validation_0-auc:0.94074
[4]	validation_0-auc:0.94074
[5]	validation_0-auc:0.94074
[6]	validation_0-auc:0.94074
[7]	validation_0-auc:0.94074
[8]	validation_0-auc:0.94074
[9]	validation_0-auc:0.94074
[10]	validation_0-auc:0.94074
[11]	validation_0-auc:0.94074
[12]	validation_0-auc:0.94074
[0]	validation_0-auc:0.77667
[1]	validation_0-auc:0.83167
[2]	validation_0-auc:0.83167
[3]	validation_0-auc:0.83167
[4]	validation_0-auc:0.83167
[5]	validation_0-auc:0.83167
[6]	validation_0-auc:0.83167
[7]	validation_0-auc:0.83167
[8]	validation_0-auc:0.83167
[9]	validation_0-auc:0.83167
[10]	validation_0-auc:0.83167
[0]	validation_0-auc:0.86727
[1]	validation_0-auc:0.87455
[2]	validation_0-auc:0.87818
[3]	validation_0-auc:0.87818
[4]	validation_0-auc:0.87818
[5]	validation_0-auc:0.87818
[6]	validation_0-auc:0.87818
[7]	validation_0-auc:0.87818
[8]	validation_0-auc:0.87818
[9]	validation_0-auc:0.87818
[10]	valid

○ メモ  
逐次concatenateする場合繰り返しconcatenateのオーバーヘッドがあるので、  
リストでまとめてconcatするよりも時間がかかるらしい。（大規模データの場合）

In [7]:
%%time
X_pd = X.to_pandas()
y_pd = y.to_pandas()
# 予測値を入れる初期の空の配列を作成
result_2 = np.empty((0, 2))

# Layer1
for train_index, val_index in cv.split(X_pd ,y_pd):
    X_train, X_val = X_pd.iloc[train_index], X_pd.iloc[val_index]
    y_train, y_val = y_pd.iloc[train_index], y_pd.iloc[val_index]
    
    # モデル学習
    eval_set = [(X_val, y_val)]
    estimator.fit(X_train, y_train, eval_set=eval_set, verbose=True)
    # 検証データ(学習に使っていないデータ)に対する予測値算出
    y_pred_proba = estimator.predict_proba(X_val)

    # 予測値を追加していく
    result_2 = np.concatenate((result, y_pred_proba))
    

[0]	validation_0-auc:0.85264
[1]	validation_0-auc:0.85382
[2]	validation_0-auc:0.90184
[3]	validation_0-auc:0.86733
[4]	validation_0-auc:0.86838
[5]	validation_0-auc:0.86838
[6]	validation_0-auc:0.86838
[7]	validation_0-auc:0.86838
[8]	validation_0-auc:0.86838
[9]	validation_0-auc:0.86798
[10]	validation_0-auc:0.86838
[11]	validation_0-auc:0.86825
[0]	validation_0-auc:0.82661
[1]	validation_0-auc:0.82990
[2]	validation_0-auc:0.82910
[3]	validation_0-auc:0.82896
[4]	validation_0-auc:0.82896
[5]	validation_0-auc:0.82923
[6]	validation_0-auc:0.82937
[7]	validation_0-auc:0.82977
[8]	validation_0-auc:0.82977
[9]	validation_0-auc:0.82795
[10]	validation_0-auc:0.82795
[0]	validation_0-auc:0.87184
[1]	validation_0-auc:0.87270
[2]	validation_0-auc:0.87385
[3]	validation_0-auc:0.87457
[4]	validation_0-auc:0.87514
[5]	validation_0-auc:0.87514
[6]	validation_0-auc:0.87514
[7]	validation_0-auc:0.87514
[8]	validation_0-auc:0.87514
[9]	validation_0-auc:0.88182
[10]	validation_0-auc:0.88182
[11]	valid

## 実装

### - 改善前
下記は最初にスクラッチで実装したときのコード。  
CVによってX,yの元のインデックスから変わってしまうことに気づかずに進めてしまった。  
X_valに対するpredict_probaの結果をそのままリストに追加してconcatすると、  
numpyに合わせてインデックスが０からふり直されてしまうことになる。  
これを正しく行うには元々のpandasのインデックスを保持したまま予測結果を格納する必要がある。  
また、XGB,LightGBMのearly stoppingのためだけにif文を実行するのは冗長な気がするので、  
early stoppingはあきらめてもよいかも。  

In [111]:
class MyStackingClassifierCV:
    def __init__(self, estimators, final_estimator=None, cv=None):
        self.estimators = estimators
        self.final_estimator = final_estimator
        self.cv = cv

    def fit(self, X, y):
        # CVでの学習済みモデルからの予測結果格納用（Layer2学習用）
        self.y_pred_dict_for_layer2 = {}
        # テストデータに対する予測のためのモデル格納用
        self.estimators_for_test = {}

        # Layer1の学習    
        for estimator_name, estimator in self.estimators:
            # モデル名と予測値のリストを対応させる。
            self.y_pred_dict_for_layer2[estimator_name] = []
            # テストデータに対するLayer1での予測値格納用

            # Layer2へ渡す特徴量生成のための学習
            for train_index, val_index in self.cv.split(X):
                # 学習用データと予測値算出用データに分ける
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                
                if estimator_name == 'XGB':
                    # XGBoostのモデル学習
                    eval_set = [(X_val, y_val)]
                    estimator.fit(X_train, y_train, eval_set=eval_set, verbose=True)
                elif estimator_name == 'LGBM':
                    # LightGBMのモデル学習
                    eval_set = [(X_val, y_val)]
                    callbacks = [lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation()]
                    estimator.fit(X_train, y_train, eval_set=eval_set, callbacks=callbacks)
                else:
                    # 決定木のモデル学習
                    estimator.fit(X_train, y_train)

                # 学習に使わなかったデータに対する予測値を算出し、リストに追加
                self.y_pred_dict_for_layer2[estimator_name].append(estimator.predict_proba(X_val))

            # 全foldでの予測値を結合してそのモデルの最終的な予測値を算出
            # !!! 結合した結果は元の入力データX,yと順番は異なってしまっていることに注意(CV時にシャッフルしているため)。
            self.y_pred_dict_for_layer2[estimator_name] = np.concatenate(self.y_pred_dict_for_layer2[estimator_name])

            # テストデータに対する予測のための学習
            if estimator_name == 'XGB':
                # XGBoostのモデル学習
                X_train2, X_val2, y_train2, y_val2 = train_test_split(X, y, test_size=0.3, random_state=0)
                eval_set = [(X_val2, y_val2)]
                self.estimators_for_test[estimator_name] = estimator.fit(X_train2, y_train2, eval_set=eval_set, verbose=True)
            elif estimator_name == 'LGBM':
                # LightGBMのモデル学習
                X_train2, X_val2, y_train2, y_val2 = train_test_split(X, y, test_size=0.3, random_state=0)
                eval_set = [(X_val2, y_val2)]
                callbacks = [lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation()]
                self.estimators_for_test[estimator_name] = estimator.fit(X_train2, y_train2, eval_set=eval_set, callbacks=callbacks)
            else:
                # 決定木のモデル学習
                self.estimators_for_test[estimator_name] = estimator.fit(X, y)
        
        # Layer1での予測値をまとめる（Layer2へ渡す用）。array[[モデル1の予測結果×2列, モデル2の予測結果×2列・・・]]の形式。
        # concatenateで横に結合していく。予測結果が×２列になるのはpredict_probaの結果だから。
        self.result_layer1 = np.concatenate(list(self.y_pred_dict_for_layer2.values()), axis=1)

        # Layer2の学習。元々の特徴量＋layer1の結果を特徴量とする。
        X_train_layer2 = np.concatenate([X, self.result_layer1], axis=1)
        self.final_estimator.fit(X_train_layer2, y)

        #return self.y_pred_dict_for_layer2
        #return self.result_layer1

    
    def predict_proba(self, X_test):
        # Layer1での学習済みモデルを使ってテストデータに対して予測
        # result_layer1_for_testの予測値・モデルの並びとresult_layer1の並びは同じなので、そのままlayer2に渡してOK
        result_layer1_for_test = [estimator.predict_proba(X_test) for _, estimator in self.estimators_for_test.items()]
        #print(result_layer1_for_test)
        result_layer1_for_test = np.concatenate(result_layer1_for_test, axis=1)
        #print(result_layer1_for_test)
        
        # テストデータに対する最終的な予測（Layer2）。元々の特徴量＋layer1の結果を特徴量とする。
        X_test_layer2 = np.concatenate([X_test, result_layer1_for_test], axis=1)
        result = self.final_estimator.predict_proba(X_test_layer2)

        return result


In [127]:
# 呼び出し側
cv = KFold(n_splits=5, shuffle=True, random_state=0)

estimators = [
    ('DT', tree.DecisionTreeClassifier(max_depth=2)),
    ('XGB', XGBClassifier(early_stopping_rounds=10, learning_rate=0.01, eval_metric='auc',random_state=0)),
    ('LGBM', lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0))
]
final_estimator = LogisticRegression()

# スタッキングのインスタンス生成
model = MyStackingClassifierCV(estimators=estimators, final_estimator=final_estimator,cv=cv)

In [113]:
model.fit(X_train, y_train)

[0]	validation_0-auc:0.80303
[1]	validation_0-auc:0.82121
[2]	validation_0-auc:0.82121
[3]	validation_0-auc:0.82121
[4]	validation_0-auc:0.82121
[5]	validation_0-auc:0.82121
[6]	validation_0-auc:0.81212
[7]	validation_0-auc:0.81212
[8]	validation_0-auc:0.78182
[9]	validation_0-auc:0.75152
[10]	validation_0-auc:0.73939
[0]	validation_0-auc:0.81818
[1]	validation_0-auc:0.81818
[2]	validation_0-auc:0.81818
[3]	validation_0-auc:0.81818
[4]	validation_0-auc:0.82955
[5]	validation_0-auc:0.82955
[6]	validation_0-auc:0.82955
[7]	validation_0-auc:0.82955
[8]	validation_0-auc:0.82955
[9]	validation_0-auc:0.81250
[10]	validation_0-auc:0.81250
[11]	validation_0-auc:0.81250
[12]	validation_0-auc:0.81250
[13]	validation_0-auc:0.81250
[0]	validation_0-auc:0.83333
[1]	validation_0-auc:0.85417
[2]	validation_0-auc:0.85417
[3]	validation_0-auc:0.85417
[4]	validation_0-auc:0.85417
[5]	validation_0-auc:0.85417
[6]	validation_0-auc:0.85417
[7]	validation_0-auc:0.85417
[8]	validation_0-auc:0.85417
[9]	valid

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [114]:
y_pred_proba = model.predict_proba(X_test)

In [116]:
y_pred_proba[:5]

array([[0.38508259, 0.61491741],
       [0.52881099, 0.47118901],
       [0.10296182, 0.89703818],
       [0.10842325, 0.89157675],
       [0.44995687, 0.55004313]])

In [117]:
y_test[:5]

94     1
18     0
33     1
98     1
181    1
Name: survived, dtype: int64

In [99]:
roc_auc_score(y_test, y_pred_proba[:, 1])

0.9102564102564104

一応、学習・予測はできたが、  
インデックスの取り扱いは間違えているので修正が必要。

### - 改善後
CVによる元のインデックスからの変更を加味して、  
元データと同じサイズの空のnumpy配列を用意してそこにval_indexごとの予測結果を入れていくようにした。  
（Polarsばかり使っているとインデックスを意識しなくなるが、CV時の学習ではちゃんと意識すること。  ）  
また、XGBoost,LGBMのearly stoppingもなしにして汎用的にfitできるように書き換えている。  
estimator_name,y_valは使ってないので不要かも。

In [179]:
class MyStackingClassifierCV_mod:
    def __init__(self, estimators, final_estimator=None, cv=None):
        self.estimators = estimators
        self.final_estimator = final_estimator
        self.cv = cv

    def fit(self, X, y):
        # ------- Layer1の学習 ---------

        # 新しい特徴量(Layer1の出力)を格納するための変数定義。（学習データ数、モデル数）というシェイプの０行列。
        new_features_train = np.zeros((X.shape[0], len(self.estimators)))

        for i, (estimator_name, estimator) in enumerate(self.estimators):

            # Layer2へ渡す特徴量生成のための学習
            for train_index, val_index in self.cv.split(X):
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]    

                # モデルの学習
                estimator.fit(X_train, y_train)

                # 学習に使わなかったデータに対する予測値を算出し、新しい特徴量として格納
                # predict_probaの１列目だけあればよいので[:, 1]とする
                # val_indexの箇所にX_valの予測結果を格納することにより、元データの順序は守られる。
                new_features_train[val_index, i] = estimator.predict_proba(X_val)[:, 1]

            # テストデータに対する予測のための学習
            # 全学習データを使って学習する。学習結果のモデルはself.estimatorsに反映されているので、別途格納する必要なし。
            # （CVでの学習モデルが上書きされる形になる）
            estimator.fit(X, y)
        
        # ------- Layer2の学習 ---------
        # layer1の結果を特徴量とする。（元々の特徴量を加味してもOK）
        self.final_estimator.fit(new_features_train, y)

        # チェック用にLayer1の出力結果を返す
        return new_features_train


    def predict_proba(self, X_test):    
        # 新しい特徴量(Layer1の出力)を格納するための変数定義（テストデータに対する予測用）。
        new_features_test = np.zeros((X_test.shape[0], len(self.estimators)))
        # Layer1での全データ学習済みモデルを使ってテストデータに対して予測
        for i, (estimator_name, estimator) in enumerate(self.estimators):
            new_features_test[:, i] = estimator.predict_proba(X_test)[:, 1]
        
        # テストデータに対する最終的な予測（Layer2）。layer1の結果を特徴量とする。
        result = self.final_estimator.predict_proba(new_features_test)

        return result




In [188]:
# 呼び出し側
cv = KFold(n_splits=5, shuffle=True, random_state=0)

estimators = [
    ('DT', tree.DecisionTreeClassifier(max_depth=2,random_state=0)),
    ('rf', RandomForestClassifier(random_state=0)),
    ('XGB', XGBClassifier(learning_rate=0.01, eval_metric='auc',random_state=0)),
    ('LGBM', lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0))
]
final_estimator = LogisticRegression()

# スタッキングのインスタンス生成
model = MyStackingClassifierCV_mod(estimators=estimators, final_estimator=final_estimator,cv=cv)

In [189]:
model.fit(X_train, y_train)

array([[0.40425532, 0.35      , 0.55150753, 0.58320055],
       [0.40425532, 0.7       , 0.48769337, 0.54981536],
       [0.97959184, 1.        , 0.7927593 , 0.85259534],
       [0.92857143, 0.91      , 0.76354849, 0.765261  ],
       [0.40425532, 0.73      , 0.51422304, 0.44884073],
       [0.92      , 0.82      , 0.71909469, 0.75841987],
       [0.44230769, 0.36      , 0.25313053, 0.42866115],
       [0.45652174, 0.57      , 0.60138512, 0.46196513],
       [0.45652174, 0.39      , 0.26547787, 0.48403298],
       [0.45945946, 0.21      , 0.5524525 , 0.53476594],
       [0.45652174, 0.32      , 0.47820899, 0.47676944],
       [0.44230769, 0.73      , 0.66333312, 0.5599008 ],
       [0.25      , 0.61      , 0.4564794 , 0.75382181],
       [0.44230769, 0.79      , 0.58188397, 0.5355605 ],
       [0.34883721, 0.63      , 0.61693907, 0.56261661],
       [0.40425532, 0.8       , 0.54500741, 0.44884073],
       [0.97959184, 1.        , 0.7927593 , 0.88593721],
       [0.40425532, 0.43      ,

In [190]:
y_pred_proba = model.predict_proba(X_test)

In [191]:
y_pred_proba

array([[0.43423151, 0.56576849],
       [0.56616054, 0.43383946],
       [0.18996141, 0.81003859],
       [0.1435601 , 0.8564399 ],
       [0.48843374, 0.51156626],
       [0.66581814, 0.33418186],
       [0.53790878, 0.46209122],
       [0.18702325, 0.81297675],
       [0.15088874, 0.84911126],
       [0.14445561, 0.85554439],
       [0.58047206, 0.41952794],
       [0.15753415, 0.84246585],
       [0.15041664, 0.84958336],
       [0.14445561, 0.85554439],
       [0.38785889, 0.61214111],
       [0.48502238, 0.51497762],
       [0.37375941, 0.62624059],
       [0.15114605, 0.84885395],
       [0.14261931, 0.85738069],
       [0.50422591, 0.49577409],
       [0.16316144, 0.83683856],
       [0.27403816, 0.72596184],
       [0.14442245, 0.85557755],
       [0.15405627, 0.84594373],
       [0.21628915, 0.78371085],
       [0.49889772, 0.50110228],
       [0.46606927, 0.53393073],
       [0.16659721, 0.83340279],
       [0.51671636, 0.48328364],
       [0.47924321, 0.52075679],
       [0.

In [192]:
roc_auc_score(y_test, y_pred_proba[:, 1])

0.8669871794871795